In [48]:
# Importing Packages

import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [49]:
# Reading the dataset
data_tests = pd.read_csv('Country-wise_data_testing_information/India_data.csv')
data_policy = pd.read_excel('Cases_and_policies_Country_wise/INDIA_Covid_Cases_and_Policies.xlsx')

In [50]:
data_policy = data_policy.drop(['Unnamed: 0'], axis = 1)

In [51]:
# Converting the datatype of dates to datetime
dates = pd.to_datetime(data_policy['day'], format = '%Y-%m-%d')
# Adding dates to the dataframe
data_policy['Date'] = dates

# Reading the dataset
#data = pd.read_csv('Country-wise_data_testing_information/India_data.csv')

# Converting the datatype of dates to datetime
dates = pd.to_datetime(data_tests['dateRep'], format = '%d-%m-%Y')

# Adding dates to the dataframe
data_tests['Date'] = dates

# Removing the original object datatype
# data = data.drop(labels = ['day'], axis = 1)

In [52]:
# Dataset Preprocessing before running the model
def dataset_preprocess_policy(df):
    
    # Setting date as the index of the dataframe
    df = df.set_index('Date')
    
    #Creating different columns with regressor variables
#     ds = df.index.to_series()
#     df['Day'] = ds.dt.day
#     df['Month'] = ds.dt.month
#     df['Weekday'] = ds.dt.dayofweek
#     df['Year'] = ds.dt.year
    
    # Dropping the text columns which have same entries at each row
    df = df.drop(['Country', 'day', 'Country Name', 'Region'], axis = 1)
    #df = df.drop(['countriesAndTerritories', 'geoId', 'day', 'countryterritoryCode'], axis = 1)
    df = df.drop(df.index[len(df)-1])
    
    #return the dataframe
    return(df)

In [53]:
# Dataset Preprocessing before running the model
def dataset_preprocess_tests(df):
    
    # Setting date as the index of the dataframe
    df = df.set_index('Date')
    
    #Creating different columns with regressor variables
#     ds = df.index.to_series()
#     df['Day'] = ds.dt.day
#     df['Month'] = ds.dt.month
#     df['Weekday'] = ds.dt.dayofweek
#     df['Year'] = ds.dt.year
    
    # Dropping the text columns which have same entries at each row
    #df = df.drop(['Country', 'day', 'Country Name', 'Region'], axis = 1)
    df = df.drop(['countriesAndTerritories', 'geoId', 'dateRep', 'countryterritoryCode'], axis = 1)
    df = df.drop(df.index[len(df)-1])
    
    #return the dataframe
    return(df)

In [54]:
data_policy = dataset_preprocess_policy(data_policy)
data_tests = dataset_preprocess_tests(data_tests)

In [55]:
data_policy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67 entries, 2020-01-31 to 2020-04-06
Data columns (total 16 columns):
Deaths                                    67 non-null int64
Cumulative Deaths                         67 non-null int64
Confirmed                                 67 non-null int64
Cumulative Confirmed                      67 non-null int64
Lockdown                                  67 non-null int64
Public Health Measures                    67 non-null int64
Governance and socio-economic measures    67 non-null int64
Humanitarian exemption                    67 non-null int64
Social distancing                         67 non-null int64
Movement restrictions                     67 non-null int64
Lockdown.1                                67 non-null float64
Public Health Measures.1                  67 non-null float64
G and S                                   67 non-null float64
H E                                       67 non-null float64
SD                           

In [56]:
data_tests.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2019-12-31 to 2020-04-09
Data columns (total 9 columns):
day                         100 non-null int64
month                       100 non-null int64
year                        100 non-null int64
cases                       100 non-null int64
deaths                      100 non-null int64
popData2018                 100 non-null int64
Daily_tests                 100 non-null float64
Daily_tests_per_thousand    100 non-null float64
Cum_tests_per_thousand      100 non-null float64
dtypes: float64(3), int64(6)
memory usage: 7.8 KB


In [57]:
data = data_policy.join(data_tests)

In [60]:
data.head(100)

,Deaths,Cumulative Deaths,Confirmed,Cumulative Confirmed,Lockdown,Public Health Measures,Governance and socio-economic measures,Humanitarian exemption,Social distancing,Movement restrictions,...,Mr,day,month,year,cases,deaths,popData2018,Daily_tests,Daily_tests_per_thousand,Cum_tests_per_thousand
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,0,0,1,1,0,2,0,0,0,0,...,0.0,31.0,1.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-01,0,0,0,1,0,4,0,0,0,0,...,0.0,1.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-02,0,0,0,1,0,6,0,0,0,0,...,0.0,2.0,2.0,2020.0,1.0,0.0,1.352617e+09,1.0,0.000000,0.000000
2020-02-03,0,0,2,3,0,8,0,0,0,0,...,0.0,3.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-04,0,0,0,3,0,10,0,0,0,0,...,0.0,4.0,2.0,2020.0,1.0,0.0,1.352617e+09,1.0,0.000000,0.000000
2020-02-05,0,0,0,3,0,12,0,0,0,0,...,0.0,5.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-06,0,0,0,3,0,14,0,0,0,0,...,0.0,6.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-07,0,0,0,3,0,16,0,0,0,0,...,0.0,7.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000
2020-02-08,0,0,0,3,0,18,0,0,0,0,...,0.0,8.0,2.0,2020.0,0.0,0.0,1.352617e+09,0.0,0.000000,0.000000


In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 67 entries, 2020-01-31 to 2020-04-06
Data columns (total 25 columns):
Deaths                                    67 non-null int64
Cumulative Deaths                         67 non-null int64
Confirmed                                 67 non-null int64
Cumulative Confirmed                      67 non-null int64
Lockdown                                  67 non-null int64
Public Health Measures                    67 non-null int64
Governance and socio-economic measures    67 non-null int64
Humanitarian exemption                    67 non-null int64
Social distancing                         67 non-null int64
Movement restrictions                     67 non-null int64
Lockdown.1                                67 non-null float64
Public Health Measures.1                  67 non-null float64
G and S                                   67 non-null float64
H E                                       67 non-null float64
SD                           